## Transformation TIF vers JPG et géolocalisation vers pixel

In [1]:
from osgeo import gdal
from skimage.io import imread
from PIL import Image
import PIL
from pathlib import Path
import pathlib
import json
from scripts.bbox import *

PIL.Image.MAX_IMAGE_PIXELS = 500000000

def get_filenames_of_path(path: pathlib.Path, ext: str = "*"):
    """
    Returns a list of files in a directory/path. Uses pathlib.
    """
    filenames = [file for file in path.glob(ext) if file.is_file()]
    assert len(filenames) > 0, f"No files found in path: {path}"
    return filenames

def world2Pixel(geoMatrix, x , y):
    u1X = geoMatrix[0]
    print(u1X)
    u1Y = geoMatrix[3]
    print(u1Y)
    xDist = geoMatrix[1]
    print(xDist)
    yDist = geoMatrix[5]
    print(yDist)
    rtnX = geoMatrix[2]
    rtnY = geoMatrix[4]
    pixel = int((x - u1X) / xDist)
    line = int((y - u1Y) /yDist)
    return (pixel, line)

def Pixel2world(geoMatrix, x, y):
    ulX = geoMatrix[0]
    ulY = geoMatrix[3]
    xDist = geoMatrix[1]
    yDist = geoMatrix[5]
    coorX = (ulX + (x * xDist))
    coorY = (ulY + (y * yDist))
    return (coorX, coorY)


## Ouverture et conversion d'images TIF (pour l'inférence)
Mettre les images TIF dans le dossier 'tifs' les sorties arriveront dans le dossier 'jpg'.

In [4]:
#Ouverture des images TIF et sauvegarde au format jpeg
directory=pathlib.Path('Transformation/tif_to_jpg/tifs')
images = get_filenames_of_path(directory)
for path in images:
    im = Image.open(path)
    rgb_im = im.convert('RGB')
    rgb_im.save('Transformation/tif_to_jpg/jpg/'+path.stem +'.jpg')

## Transformation de jsons contenant les coordonnées en pixel en json contenant les coordonnées en géolocalisation.
Rentrer les jsons dans le dossier pixel_to_world/jsons, et les images TIF correspondantes (portant le même nom à l'extension prêt) dans le dossier pixel_to_world/tifs. 

Les résultats seront enregistrés dans le dossier pixel_to_world/output et les fichiers json auront les mêmes noms que les json correspondants.

Le json en sortie est de la forme:

json = {"boxes": [[[xmin, ymin],[xmax,ymax]],...], "labels": [bird, bird, bird]}

Donc json['boxes'][i] a la classe json['labels'][i]

In [14]:
#Pixel vers location avec json et images tif
labels=['bird'] # rentrer les classes ici, dans le même ordre que pour l'entrainement
mapping={}
for i in range(len(labels)):
    mapping[str(i+1)]=labels[i]

directory_jsons=pathlib.Path('Transformation/pixel_to_world/jsons')
directory_tifs=pathlib.Path('Transformation/pixel_to_world/tifs')
jsons = get_filenames_of_path(directory_jsons)
tifs = get_filenames_of_path(directory_tifs)
dic={}
dic['boxes']=[]
dic['labels']=[]

for path in jsons:
    tif = 'Transformation/' + get_tif_path_ptw(path)
    msg = "en cours de transformation: "+ path.stem + '.json'
    print(msg)
    src = gdal.Open(tif)
    geoTrans = src.GetGeoTransform()
    with open(path) as f:
        data = json.load(f)
    for i in range(len(data['boxes'])):
        box=data['boxes'][i]
        label=data['labels'][i]
        box_geo=[]
        xmin,ymin,xmax,ymax=box[0],box[1],box[2],box[3]
        p1 = Pixel2world(geoTrans, xmin, ymin)
        p2 = Pixel2world(geoTrans, xmax, ymax)
        box_geo.append(p1)
        box_geo.append(p2)
        dic['boxes'].append(box_geo)
        dic['labels'].append(mapping[str(label)])
    
    with open('Transformation/pixel_to_world/output/'+path.stem+'.json', 'w') as f:
        json.dump(dic, f)

en cours de transformation: 000.json


## Transformation de jsons contenant les coordonnées en géolocalisatoin en json contenant les coordonnées en pixel.
Rentrer les jsons dans le dossier world_to_pixel/jsons, et les images TIF correspondantes (portant le même nom à l'extension prêt) dans le dossier world_to_pixel/tifs. 

Les résultats seront enregistrés dans le dossier world_to_pixel/output et les fichiers json auront les mêmes noms que les json correspondants.

Bien préciser la/les classes souhaitées.

In [18]:
#Location vers pixel avec image tif et geojson
objectif='oiseau' # 'oiseau', 'nid' ou 'espece'

directory_jsons=pathlib.Path('Transformation/world_to_pixel/jsons')
directory_tifs=pathlib.Path('Transformation/world_to_pixel/tifs')
geojsons = get_filenames_of_path(directory_jsons)
tifs = get_filenames_of_path(directory_tifs)


for path in geojsons:
    msg = "en cours de transformation: "+ path.stem + '.json'
    print(msg)
    dic={}
    dic['boxes']=[]
    dic['labels']=[]
    tif = 'Transformation/' + get_tif_path_wtp(path)
    src = gdal.Open(tif)
    geoTrans = src.GetGeoTransform()
    with open(path) as f:
        data = json.load(f)
    for i in data['features']:
        box=get_pixel_coordinates(i['geometry']['coordinates'][0][0],geoTrans)
        label=get_label(objectif,i['properties'])
        dic['boxes'].append(box)
        dic['labels'].append(label)
    
    with open('Transformation/world_to_pixel/output/' + path.stem + '.json', 'w') as f:
        json.dump(dic, f)

en cours de transformation: bounding_box_K3K1.json
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.01

1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760

0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.0107600003

0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.010760000362301458
1222937.2987
7190303.0718
0.010760000362301458
-0.0107600003